<a href="https://colab.research.google.com/github/gpugems/llm_demo/blob/main/class_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv openai

In [19]:
import os
import openai

from openai import OpenAI
from google.colab import userdata

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')



In [32]:
Client = OpenAI()



completion = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are ChatGPT, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."},
        {"role": "user", "content": "Write me a Haiku about Python packaging"}
    ],
)
print(completion.choices[0].message.content)



Code in neat bundles,  
Packages wrapped with intent,  
Python's smooth journey.
